[1] This code is to create dataset for inference. <br> - The original image is simply tiled without any labeling, and padding is applied only to the edges of individual tiles.
- Cropped the images into 1024*1024 for YOLO
- 12500*12500 images are not divided by 1024*1024. Therefore, padding is applied (same as our training dataset)

In [ ]:
import os
import rasterio
from rasterio.windows import Window
from rasterio.transform import from_origin
import numpy as np
from tqdm import tqdm

def save_padded_tile(src, x, y, tile_size, out_path):
    # Calculate actual width/height for edge tiles
    width = min(tile_size, src.width - x)
    height = min(tile_size, src.height - y)

    # Define window and read actual data
    window = Window(x, y, width, height)
    data = src.read(window=window)

    # Create padded array with zeros
    padded = np.zeros((src.count, tile_size, tile_size), dtype=src.dtypes[0])
    padded[:, :height, :width] = data

    # Define correct transform using tile's upper-left pixel
    origin_x, origin_y = src.transform * (x, y)
    transform = from_origin(origin_x, origin_y, src.res[0], src.res[1])

    # Update profile for output
    profile = src.profile.copy()
    profile.update({
        "height": tile_size,
        "width": tile_size,
        "transform": transform
    })

    # Write the padded tile
    with rasterio.open(out_path, "w", **profile) as dst:
        dst.write(padded)

def split_images_with_padding(tif_path, out_img_dir, tile_size=1024):
    os.makedirs(out_img_dir, exist_ok=True)
    img_name = os.path.splitext(os.path.basename(tif_path))[0]

    with rasterio.open(tif_path) as src:
        for y in range(0, src.height, tile_size):
            for x in range(0, src.width, tile_size):
                tile_name = f"{img_name}_tile_{x}_{y}.tif"
                out_path = os.path.join(out_img_dir, tile_name)

                if os.path.exists(out_path):
                    continue

                save_padded_tile(src, x, y, tile_size, out_path)

def main():
    tif_dir = "/shared/data/climateplus2025/CapeTown_Image_2023"
    out_img_dir = "/shared/data/climateplus2025/Prediction_for_poster_3_images_July21/CapeTown_Image_2023_tiles_1024_for_prediction"
    tile_size = 1024

    os.makedirs(out_img_dir, exist_ok=True)

    # Selected image names (no file extension)
    selected_image_names = [
        '2023_RGB_8cm_W25C_16',
        '2023_RGB_8cm_W24A_17',
        '2023_RGB_8cm_W57B_8'
    ]

    # Map all .tif files in tif_dir
    tif_map = {}
    for root, dirs, files in os.walk(tif_dir):
        for f in files:
            if f.endswith(".tif"):
                name = os.path.splitext(f)[0]
                full_path = os.path.join(root, f)
                tif_map[name] = full_path

    # Collect full paths to selected .tif files
    tif_files = []
    for name in selected_image_names:
        if name in tif_map:
            tif_files.append(tif_map[name])
        else:
            print(f"[!] Missing file: {name}.tif")

    # Process each file
    for tif_path in tqdm(tif_files, desc="Tiling selected images with padding"):
        split_images_with_padding(tif_path, out_img_dir, tile_size)

    print("Finished tiling selected images.")

if __name__ == "__main__":
    main()


Tiling selected images with padding: 100%|██████████| 3/3 [00:43<00:00, 14.66s/it]

Finished tiling selected images.


[2] Following processes are set up to measure model performance <br>
- Extract labels for ground truth matching for YOLO
- This script is used to generate the training dataset. It splits the image into tiles and creates the corresponding YOLO label (.txt) files.
The labels are read from a GeoPackage, and each polygon is converted into a YOLO-style rotated bounding box. (Used for performance evaluation)

In [ ]:
import glob
import os
import numpy as np
import rasterio
from rasterio.windows import Window
from shapely.geometry import box
import geopandas as gpd
import cv2
from tqdm import tqdm

def sort_points_clockwise(pts):
    center = np.mean(pts, axis=0)
    angles = np.arctan2(pts[:, 1] - center[1], pts[:, 0] - center[0])
    return pts[np.argsort(angles)]

def fix_invalid_geometry(g):
    try:
        if not g.is_valid:
            return g.buffer(0)
        return g
    except:
        return None

def save_tile_and_label(tile_img, window_transform, label_gdf, tile_name, out_img_dir, out_lbl_dir, tile_size=1024):
    out_img_path = os.path.join(out_img_dir, f"{tile_name}.tif")
    out_lbl_path = os.path.join(out_lbl_dir, f"{tile_name}.txt")

    # Save image
    profile = {
        "driver": "GTiff",
        "dtype": tile_img.dtype,
        "count": tile_img.shape[0],
        "height": tile_size,
        "width": tile_size,
        "transform": window_transform,
        "crs": label_gdf.crs
    }
    with rasterio.open(out_img_path, "w", **profile) as dst:
        dst.write(tile_img)

    # Get tile bounds
    tile_bounds = box(*window_transform * (0, 0), *window_transform * (tile_size, tile_size))
    tile_bounds = fix_invalid_geometry(tile_bounds)
    anns = label_gdf[label_gdf.geometry.intersects(tile_bounds)].copy()
    if anns.empty:
        return

    anns['geometry'] = anns['geometry'].apply(fix_invalid_geometry)
    anns = anns[anns.geometry.notnull()]
    anns['geometry'] = anns.geometry.intersection(tile_bounds)
    anns = anns[~anns.geometry.is_empty]

    label_lines = []

    for _, row in anns.iterrows():
        try:
            if row['PV_normal'] == 1 or row['PV_heater'] == 1 or row['PV_pool'] == 1:
                class_id = 0
            else:
                continue

            geom = row.geometry
            if geom.geom_type == "MultiPolygon":
                geom = max(geom.geoms, key=lambda g: g.area)
            elif geom.geom_type != "Polygon":
                continue

            coords = np.array(list(geom.exterior.coords[:-1]), dtype=np.float32)
            pixel_coords = np.array([~window_transform * (lon, lat) for lon, lat in coords], dtype=np.float32)

            if len(pixel_coords) < 3 or np.any(np.isnan(pixel_coords)) or np.any(np.isinf(pixel_coords)):
                continue

            if cv2.contourArea(pixel_coords) < 1.0:
                continue

            rect = cv2.minAreaRect(pixel_coords)
            box_pts = cv2.boxPoints(rect)
            box_pts = sort_points_clockwise(box_pts)

            if cv2.contourArea(box_pts) < 1.0:
                continue

            box_pts[:, 0] /= tile_size
            box_pts[:, 1] /= tile_size
            box_pts = np.clip(box_pts, 0, 1)

            if box_pts.shape != (4, 2):
                continue

            coords_str = " ".join([f"{pt[0]:.6f} {pt[1]:.6f}" for pt in box_pts])
            label_lines.append(f"{class_id} {coords_str}")

        except Exception as e:
            print(f"[!] Error in {tile_name}: {e}")
            continue

    if label_lines:
        with open(out_lbl_path, "w") as f:
            f.write("\n".join(label_lines))
        print(f"[✓] Saved {tile_name}.tif with {len(label_lines)} labels")

def process_selected_images(tif_dir, label_gdf, out_img_dir, out_lbl_dir, tile_size=1024):
    os.makedirs(out_img_dir, exist_ok=True)
    os.makedirs(out_lbl_dir, exist_ok=True)

    all_tif_paths = glob.glob(os.path.join(tif_dir, "**", "*.tif"), recursive=True)
    tif_dict = {os.path.splitext(os.path.basename(p))[0]: p for p in all_tif_paths}
    selected_image_names = label_gdf["image_name"].unique()

    for name in tqdm(selected_image_names, desc="Processing TIFFs with padding"):
        if name not in tif_dict:
            print(f"[!] TIFF not found: {name}")
            continue

        tif_path = tif_dict[name]
        gdf = label_gdf[label_gdf["image_name"] == name]
        if gdf.empty:
            print(f"[!] No annotations for: {name}")
            continue

        with rasterio.open(tif_path) as src:
            if gdf.crs != src.crs:
                gdf = gdf.to_crs(src.crs)

            img = src.read()
            _, h, w = img.shape

            pad_h = (tile_size - h % tile_size) % tile_size
            pad_w = (tile_size - w % tile_size) % tile_size

            if pad_h > 0 or pad_w > 0:
                img_padded = np.pad(img, ((0, 0), (0, pad_h), (0, pad_w)), mode='constant')
            else:
                img_padded = img

            padded_h, padded_w = img_padded.shape[1:]

            for y in range(0, padded_h, tile_size):
                for x in range(0, padded_w, tile_size):
                    tile_img = img_padded[:, y:y+tile_size, x:x+tile_size]
                    window_transform = src.transform * rasterio.Affine.translation(x, y)
                    tile_name = f"{name}_tile_{x}_{y}"
                    save_tile_and_label(tile_img, window_transform, gdf, tile_name, out_img_dir, out_lbl_dir, tile_size)

    print("Finished tiling with padding")

def main():
    tif_dir = "/shared/data/climateplus2025/CapeTown_Image_2023"
    out_img_dir = "/shared/data/climateplus2025/Prediction_for_poster_3_images_July21/CapeTown_tiles_selected/images"
    out_lbl_dir = "/shared/data/climateplus2025/Prediction_for_poster_3_images_July21/CapeTown_tiles_selected/labels"
    gpkg_path = "/shared/data/climateplus2025/Prediction_for_poster_July21/0.Image_files_selection/final_annotations_PV_all_types_balanced_3_cleaned.gpkg"

    try:
        label_gdf = gpd.read_file(gpkg_path)
        print(f"Loaded {len(label_gdf)} annotations from GPKG")
    except Exception as e:
        print(f"[!] Failed to read GPKG: {e}")
        return

    process_selected_images(tif_dir, label_gdf, out_img_dir, out_lbl_dir)

if __name__ == "__main__":
    main()


Loaded 1022 annotations from GPKG


Processing TIFFs with padding:   0%|          | 0/3 [00:00<?, ?it/s]

[✓] Saved 2023_RGB_8cm_W57B_8_tile_1024_0.tif with 3 labels
[✓] Saved 2023_RGB_8cm_W57B_8_tile_2048_0.tif with 1 labels
[✓] Saved 2023_RGB_8cm_W57B_8_tile_3072_0.tif with 1 labels
[✓] Saved 2023_RGB_8cm_W57B_8_tile_4096_0.tif with 3 labels
[✓] Saved 2023_RGB_8cm_W57B_8_tile_5120_0.tif with 2 labels
[✓] Saved 2023_RGB_8cm_W57B_8_tile_7168_0.tif with 3 labels
[✓] Saved 2023_RGB_8cm_W57B_8_tile_8192_0.tif with 5 labels
[✓] Saved 2023_RGB_8cm_W57B_8_tile_9216_0.tif with 5 labels
[✓] Saved 2023_RGB_8cm_W57B_8_tile_10240_0.tif with 6 labels
[✓] Saved 2023_RGB_8cm_W57B_8_tile_11264_0.tif with 7 labels
[✓] Saved 2023_RGB_8cm_W57B_8_tile_12288_0.tif with 7 labels
[✓] Saved 2023_RGB_8cm_W57B_8_tile_1024_1024.tif with 9 labels
[✓] Saved 2023_RGB_8cm_W57B_8_tile_3072_1024.tif with 4 labels
[✓] Saved 2023_RGB_8cm_W57B_8_tile_7168_1024.tif with 4 labels
[✓] Saved 2023_RGB_8cm_W57B_8_tile_8192_1024.tif with 2 labels
[✓] Saved 2023_RGB_8cm_W57B_8_tile_9216_1024.tif with 7 labels
[✓] Saved 2023_RGB_8cm

Processing TIFFs with padding:  33%|███▎      | 1/3 [00:13<00:26, 13.43s/it]

[✓] Saved 2023_RGB_8cm_W24A_17_tile_2048_0.tif with 2 labels
[✓] Saved 2023_RGB_8cm_W24A_17_tile_3072_0.tif with 6 labels
[✓] Saved 2023_RGB_8cm_W24A_17_tile_4096_0.tif with 9 labels
[✓] Saved 2023_RGB_8cm_W24A_17_tile_5120_0.tif with 6 labels
[✓] Saved 2023_RGB_8cm_W24A_17_tile_6144_0.tif with 10 labels
[✓] Saved 2023_RGB_8cm_W24A_17_tile_7168_0.tif with 7 labels
[✓] Saved 2023_RGB_8cm_W24A_17_tile_1024_1024.tif with 2 labels
[✓] Saved 2023_RGB_8cm_W24A_17_tile_2048_1024.tif with 12 labels
[✓] Saved 2023_RGB_8cm_W24A_17_tile_3072_1024.tif with 1 labels
[✓] Saved 2023_RGB_8cm_W24A_17_tile_4096_1024.tif with 1 labels
[✓] Saved 2023_RGB_8cm_W24A_17_tile_5120_1024.tif with 7 labels
[✓] Saved 2023_RGB_8cm_W24A_17_tile_6144_1024.tif with 1 labels
[✓] Saved 2023_RGB_8cm_W24A_17_tile_7168_1024.tif with 1 labels
[✓] Saved 2023_RGB_8cm_W24A_17_tile_0_2048.tif with 2 labels
[✓] Saved 2023_RGB_8cm_W24A_17_tile_1024_2048.tif with 7 labels
[✓] Saved 2023_RGB_8cm_W24A_17_tile_2048_2048.tif with 2 la

Processing TIFFs with padding:  67%|██████▋   | 2/3 [00:25<00:12, 12.77s/it]

[✓] Saved 2023_RGB_8cm_W24A_17_tile_7168_12288.tif with 2 labels
[✓] Saved 2023_RGB_8cm_W24A_17_tile_9216_12288.tif with 2 labels
[✓] Saved 2023_RGB_8cm_W25C_16_tile_1024_0.tif with 3 labels
[✓] Saved 2023_RGB_8cm_W25C_16_tile_2048_0.tif with 2 labels
[✓] Saved 2023_RGB_8cm_W25C_16_tile_3072_0.tif with 1 labels
[✓] Saved 2023_RGB_8cm_W25C_16_tile_4096_0.tif with 1 labels
[✓] Saved 2023_RGB_8cm_W25C_16_tile_7168_0.tif with 2 labels
[✓] Saved 2023_RGB_8cm_W25C_16_tile_10240_0.tif with 11 labels
[✓] Saved 2023_RGB_8cm_W25C_16_tile_1024_1024.tif with 2 labels
[✓] Saved 2023_RGB_8cm_W25C_16_tile_2048_1024.tif with 3 labels
[✓] Saved 2023_RGB_8cm_W25C_16_tile_3072_1024.tif with 5 labels
[✓] Saved 2023_RGB_8cm_W25C_16_tile_4096_1024.tif with 6 labels
[✓] Saved 2023_RGB_8cm_W25C_16_tile_5120_1024.tif with 6 labels
[✓] Saved 2023_RGB_8cm_W25C_16_tile_7168_1024.tif with 1 labels
[✓] Saved 2023_RGB_8cm_W25C_16_tile_10240_1024.tif with 1 labels
[✓] Saved 2023_RGB_8cm_W25C_16_tile_11264_1024.tif wi

Processing TIFFs with padding: 100%|██████████| 3/3 [00:38<00:00, 12.77s/it]

[✓] Saved 2023_RGB_8cm_W25C_16_tile_5120_12288.tif with 8 labels
[✓] Saved 2023_RGB_8cm_W25C_16_tile_7168_12288.tif with 1 labels
Finished tiling with padding
